Imports

In [ ]:
import copy
import torch
import math
from torch import nn

import matplotlib.pyplot as plt

from src.datasets.datasets import get_dataset_by_name
from src.modeling.vgg import COB_VGG
from src.utils import select_best_device, get_project_root
from src.training.train import evaluate
from src.modeling.pruning import PruningStrategy

Args

In [ ]:
train_loader, test_loader = get_dataset_by_name(
    dataset_name="cifar-10",
    batch_size=10000,
    eval_batch_size=1024,
    num_workers=0,
    download=True,
)

model_path = get_project_root() / "pretrained_weights" / "cifar_10-tsra_rms"

Functions

In [ ]:
def basic_bar_graph(y, figsize=(6,2), dpi=100, xlabel = None, ylabel = None, title = None):
    # Create bar graph
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(range(len(y)), y, width=1.0, edgecolor='none')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


def graph_importance_scores(model):
    for m in model.get_ordered_cob_modules():
        importance_scores = m._importance_scores
        basic_bar_graph(importance_scores, xlabel="Dims", ylabel="Importance")

Setup

In [ ]:
device = select_best_device()

original_model = COB_VGG.from_pretrained(model_path)
original_model.eval()
original_model.to(device)

data = next(iter(train_loader))[0]
data = data.to(device)

print()

Copy model

The copied model is used beyond this point. This way, rapid pruning testing can be done without having to wait to load the model from storage each time

In [ ]:
model = copy.deepcopy(original_model)

First eval

In [ ]:
evaluate(model, test_loader, device)[1]

Compute importance + graph

In [ ]:
model.compute_importance_scores(data)

graph_importance_scores(model)

Rotate + compute importance + graph again

In [ ]:
model.rotate_for_prune(data)
model.compute_importance_scores(data)

graph_importance_scores(model)

Prune + eval

In [ ]:
## NOTE: Supplying a tuple of TWO values to PruningStrategy indicates that the respective pruning should take place SEPARATELY within the two SEPARATE subspaces of the activation function. Supplying ONE value indicates the entire hidden space should be considered as one.
## Eg. PruningStrategy(proportion=(p,p)) vs PruningStrategy(proportion=p)

# ## Prune set % of dimensions from each layer
# p_desired_prune = 0.7
# p = 1-math.sqrt(1-p_desired_prune)
# pruning_strategy = PruningStrategy(proportion=(p,p))

## Threshold pruning methods
thres = 0.1
# pruning_strategy = PruningStrategy(zscore_cutoff=(thres, thres))
# pruning_strategy = PruningStrategy(prop_of_avg=(thres, thres))
# pruning_strategy = PruningStrategy(prop_of_med=(thres, thres))
pruning_strategy = PruningStrategy(prop_of_max=(thres, thres))



def parameter_count(model):
    return sum(p.numel() for p in model.parameters())

p_before_pruning = parameter_count(model)

for m in model.get_ordered_cob_modules():
    results = m.structured_prune(pruning_strategy)
    print("-------------------------------------------------")
    print(f"Number pruned: {results['pruned_absolute_num']}")
    print(f"Prop pruned: {results['pruned_prop']}")
    print("-------------------------------------------------")

p_after_pruning = parameter_count(model)

print(f"Parameter count before pruning: {p_before_pruning}")
print(f"Parameter count after pruning: {p_after_pruning}")
print(f"Parameter count pruned: {p_before_pruning - p_after_pruning}")
print(f"Percentage pruned: {(p_before_pruning - p_after_pruning) / p_before_pruning}")

evaluate(model, test_loader, device)[1]